In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

# import os
# os.chdir("drive/MyDrive/Colab Notebooks/Authorship Identification/")

import re
import torch
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import DistilBertTokenizer, BertTokenizer, DebertaTokenizer

random_seed = 42
np.random.seed = 42
torch.random.seed = 42

/home/ys724/.conda/envs/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ys724/.conda/envs/venv/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Load C50 Dataset (if already loaded)

- this (original dataset) is 50 train - 50 test split

In [ ]:
train_df = pd.read_csv('../authorship_identification/data/train.csv')
test_df = pd.read_csv('../authorship_identification/data/test.csv')
train_df.rename(columns={'text': 'content', 'label': 'author'}, inplace=True)
test_df.rename(columns={'text': 'content', 'label': 'author'}, inplace=True)

- this dataset is the original one with entity names removed

In [11]:
train_df = pd.read_csv('../authorship_identification/data/train_noents.csv')
test_df = pd.read_csv('../authorship_identification/data/test_noents.csv')
train_df.rename(columns={'text': 'content', 'label': 'author'}, inplace=True)
test_df.rename(columns={'text': 'content', 'label': 'author'}, inplace=True)

- this is 90 train - 10 test split

In [ ]:
test_frac = 0.1
train_df = pd.concat([train_df, test_df])
test_df = train_df.sample(frac=test_frac, random_state=random_seed)
train_df = train_df.drop(test_df.index)

### Load "All the News" 1 Dataset

In [ ]:
train_df = pd.read_csv('data/all_the_news_noents.csv')
train_df = train_df[(train_df.publication == 'Breitbart')]
top10_authors = train_df[['author','id']].groupby('author').count().sort_values(by='id', ascending=False).head(10).reset_index() # this == what the paper proposed
print('The authors are', top10_authors.author.values)
train_df = pd.merge(train_df, top10_authors[['author']], on='author', how='right')
train_df = train_df.groupby(by='author').sample(500, random_state=random_seed)
train_df.shape

### Preprocessing

In [12]:
# if C50 split 50-50
test_frac = 0

# if NOT C50 split 50-50
# test_frac = 0.15
# test_df = train_df.sample(frac=test_frac, random_state=random_seed)
# train_df = train_df.drop(test_df.index).reset_index()
# test_df = test_df.reset_index()

le = preprocessing.LabelEncoder()
le.fit(train_df['author'])
train_df['author_id'] = le.transform(train_df['author']).astype(int)
test_df['author_id'] = le.transform(test_df['author']).astype(int)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [13]:
print('The average length of text of an article in training set is', np.mean([len(content.split()) for content in train_df['content'].values]), 
      'and in test set is', np.mean([len(content.split()) for content in test_df['content'].values]))

The average length of text of an article in training set is 420.3584 and in test set is 427.668


In [14]:
valid_frac = 0.1 / (1 - test_frac)
valid_df = train_df.sample(frac=valid_frac, random_state=random_seed)
valid_index = valid_df.index
train_df = train_df.drop(valid_df.index).reset_index()
valid_df = valid_df.reset_index()

In [15]:
pd.options.display.max_colwidth = 100
tmp_df = pd.read_csv('../authorship_identification/data/train.csv').drop(valid_index).reset_index()
# tmp_df = tmp_df.merge(train_df[['Unnamed: 0']], how='right')
tmp_df['text']

0       The Czech capital market, roundly criticised for a lack of transparency, needs an independent re...
1       Sweden beat the Czech Republic 3-0 in a World Cup ice hockey game on Thursday, setting up a show...
2       The fall in the Czech trade deficit to 10.5 billion crowns in September from 14.5 billion in Aug...
3       Czech paper concern Sepap Group a.s. on Friday said its nine-month net profit fell as a shutdown...
4       Czech shares rallied on Monday following the coalition government's win in weekend Senate electi...
                                                       ...                                                 
2245    "The Times They Are a-Changin" -- Bob Dylan's counter-culture anthem of the 1960's  -- is being ...
2246    Veronika Hirsch, the flamboyant Canadian stock picker hired recently to spearhead Fidelity Inves...
2247    Toronto, Canada's biggest city and financial capital, is bracing for a near shutdown Friday when...
2248    Anti-government prot

In [ ]:
#     text = tokenizer("Commonwealth [MASK] of Australia (_) managing", padding='max_length', max_length = 32, truncation=True, return_tensors="pt")
#     print(text)
#     mask_val = tokenizer.vocab.get('[MASK]')
#     text['attention_mask'] = torch.where(text['input_ids'] == mask_val, 0, text['attention_mask'])
#     print(text)
#     text = "Westpac Banking Corp Ltd is expected to report on Tuesday that its net profit growth was reined"
#     sub = "Banking"
#     i = text.find(sub)
#     print(i, text[i-1], text[i+len(sub)])
#     if i == -1 or (i > 0 and text[i-1].isalpha()) or (i+len(sub) < len(text) and text[i+len(sub)].isalpha()):
#         print('no')
#     else:
#         print('yes')
# tokenizer.mask_token_id, tokenizer.all_special_ids, tokenizer.special_tokens_map
# tokenizer("[MASK] fheou  hewoh iwo")
# tmp = tokenizer.tokenize("In a column intended to suggest the National Rifle Association (NRA) is exaggerating Hillary")
# tmp2 = tokenizer.convert_tokens_to_string(tmp)
# tmp, tmp2

In [6]:
import re

def mask_contents(threshold, mask_token='[MASK]'):
    vectorizer = TfidfVectorizer(ngram_range=(1, 2), lowercase=False) #, stop_words='english')
    vectorizer.fit(pd.concat([train_df['content'], valid_df['content']]))
    X_train = vectorizer.transform(train_df['content'])
    X_valid = vectorizer.transform(valid_df['content'])
    X_test = vectorizer.transform(test_df['content'])
    feat_names = vectorizer.get_feature_names_out()
    feat_idf = vectorizer.idf_

    def mask_content(X, df):
        mask_words = set(feat_names[feat_idf >= threshold])
        ret = []
        for i in range(df['content'].shape[0]):
            text = df['content'][i]
            splits = re.split(r"(?u)\b\w\w+\b", text)
            tokens = re.findall(r"(?u)\b\w\w+\b", text)
            tokens_masked = []
            i = 0
            while i < len(tokens):
                t = tokens[i]
                if t in mask_words: # 1-gram
                    tokens_masked.append(mask_token)
                    i += 1
                elif i < len(tokens_masked)-1 and t + ' ' + tokens[i+1] in mask_words: # 2-gram
                    tokens_masked.append(mask_token)
                    tokens_masked.append(mask_token)
                    i += 2
                else:
                    tokens_masked.append(t)
                    i += 1
            text_masked = ''.join([a + b for a, b in zip(splits, tokens_masked)])
            ret.append(text_masked)
        return ret

    train_df['masked_content'] = mask_content(X_train, train_df)
    valid_df['masked_content'] = mask_content(X_valid, valid_df)
    test_df['masked_content'] = mask_content(X_test, test_df)

In [7]:
# for visualization ONLY
pd.options.display.max_colwidth = 100
mask_contents(threshold=2., mask_token='_')
train_df['masked_content']

0       The _ market, _ _ for a _ of _, _ an _ _, but it is not a _ for all of the market's _, a _ _ sai...
1       _ the _ 3-0 in a _ _ _ on , _ up a _ for _ _ _ and _ the _ _ _ _ for _.\nThe _-_ _ _ of the _ fr...
2       The _ in the _ _ to _ in from in _ market _, and the _ _ _ _ _ is on _, analysts said on .\n() _...
3       _ _ a.s. on said its _ _ _ as a _ of its _ _ _, _ with a _ in the _ _ _ into its _ _.\nsaid its ...
4       _ _ on _ the _ _'s _ in _, but analysts said the _ was _ more to a _ _ than to a _ in _ _.\n_ _ ...
                                                       ...                                                 
2245    "" -- _-_ _ of the 's  -- is _ _ in a _ for a _ and some _ are up in _.\nis _ a _ _ _ the _'s _ ...
2246    , the _ _ _ _ _ to _ _ to _ the market, has been _ from _ _ as _ is _ by _.\nThe _ _ -- after 's...
2247    , 's _ _ and _ _, is _ for a _ _ when _ _ the _ _ _ _ _ by 's _ _.\nThe _-_ "" on and are _ up t...
2248    _-_ _ _ _ 's _ _ on 

### Dataset Class

In [8]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = df['author_id'].values
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['masked_content']]
        # add attention mask
#         mask_val = tokenizer.vocab.get('_')
#         for text in self.texts:
#             text['attention_mask'] = torch.where(text['input_ids'] == mask_val, 0, text['attention_mask'])

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

### Model Class

In [9]:
from torch import nn
from transformers import DistilBertForSequenceClassification, BertForSequenceClassification, DebertaForSequenceClassification, get_linear_schedule_with_warmup

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super().__init__()

        self.bert = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(le.classes_))
        # self.dropout = nn.Dropout(dropout)
        # self.linear = nn.Linear(768, 50)
        # self.relu = nn.ReLU()

    def forward(self, input_id, labels=None):

        # pooled_output, = self.bert(input_ids=input_id, attention_mask=mask, return_dict=False)
        # pooled_output = pooled_output[:,0,:]
        # dropout_output = self.dropout(pooled_output)
        # linear_output = self.linear(dropout_output)
        # final_layer = self.relu(linear_output)
        # print(input_id.shape, labels.shape)
        final_layer = self.bert(input_id, labels=labels)

        return final_layer

### Training

In [ ]:
# !git clone https://gist.github.com/NTT123/4596e5533e573c8ceab2f319ab5d36a2 jslog
# import random
# import math
# import time
# from jslog.jslogger import JSLogger

# logger = JSLogger('train/valid loss', ['train', 'valid'])
# logger_ = JSLogger('train/valid accuracy', ['train', 'valid'])

In [ ]:
from torch.optim import AdamW
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

def train(model, train_data, val_data, learning_rate, epochs, batch_size):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=batch_size)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    lr_scheduler = get_linear_schedule_with_warmup(
        optimizer=optimizer,
        num_warmup_steps=100,
        num_training_steps=len(train_dataloader) * epochs,
    )

    if use_cuda:
        
        print('cuda version', torch.__version__)
        model = model.cuda()
        criterion = criterion.cuda()

    for epoch_num in range(epochs):
        
        train_loss = 0
        train_pred = []
        train_truth = []
        model.train()

        for train_input, train_label in tqdm(train_dataloader):

            train_label = train_label.to(device)
            # mask = train_input['attention_mask'].to(device)
            input_id = train_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, train_label)
                
            # batch_loss = criterion(output, train_label.long())
            batch_loss = output.loss
            train_loss += batch_loss.item()
            train_pred.append(output.logits.argmax(dim=1))
            train_truth.append(train_label)

            model.zero_grad()
            batch_loss.backward()
            optimizer.step()
            lr_scheduler.step()
                
        train_truth = torch.cat(train_truth).detach().cpu().numpy()
        train_pred = torch.cat(train_pred).detach().cpu().numpy()
        train_acc = accuracy_score(train_truth, train_pred)
        train_f1 = f1_score(train_truth, train_pred, average='macro')
            
        val_loss = 0
        val_pred = []
        val_truth = []
        model.eval()

        with torch.no_grad():

            for val_input, val_label in val_dataloader:

                val_label = val_label.to(device)
                # mask = val_input['attention_mask'].to(device)
                input_id = val_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, val_label)

                # batch_loss = criterion(output, val_label.long())
                batch_loss = output.loss
                val_loss += batch_loss.item()
                val_pred.append(output.logits.argmax(dim=1))
                val_truth.append(val_label)
            
        val_truth = torch.cat(val_truth).detach().cpu().numpy()
        val_pred = torch.cat(val_pred).detach().cpu().numpy()
        val_acc = accuracy_score(val_truth, val_pred)
        val_f1 = f1_score(val_truth, val_pred, average='macro')
            
        print('Epochs: {} | Train Loss: {:.4f} | Train Accuracy: {:.4f} | Train F1: {:.4f}'\
                .format(epoch_num + 1, train_loss, train_acc, train_f1))
        print('           | Val Loss:   {:.4f} | Val Accuracy:   {:.4f} | Val F1:   {:.4f}'\
                .format(val_loss, val_acc, val_f1))
            
        # logger.log(epoch_num + 1, {'train': total_loss_train / len(train_data), 'valid': total_loss_val / len(val_data)})
        # logger_.log(epoch_num + 1, {'train': total_acc_train / len(train_data), 'valid': total_acc_val / len(val_data)})
    return train_acc, train_f1, val_acc, val_f1

# logger.show()
# logger_.show()
EPOCHS = 15
LR = 2e-5
BATCH_SIZE = 16
model = BertClassifier()
print('There are', sum(p.numel() for p in model.parameters()), 'parameters')

mask_contents(threshold=3.)
print(train_df['masked_content'])
              
train(model, train_df, valid_df, LR, EPOCHS, BATCH_SIZE)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

There are 109520690 parameters
0       The [MASK] market, [MASK] [MASK] for a [MASK] of [MASK], [MASK] an [MASK] [MASK], but it is not ...
1       [MASK] the [MASK] 3-0 in a [MASK] [MASK] [MASK] on , [MASK] up a [MASK] for group [MASK] against...
2       The [MASK] in the [MASK] [MASK] to [MASK] in from in [MASK] market [MASK], and the [MASK] [MASK]...
3       [MASK] [MASK] a.s. on said its [MASK] profit [MASK] as a [MASK] of its [MASK] [MASK] [MASK], [MA...
4       shares [MASK] on [MASK] the [MASK] government's [MASK] in [MASK], but analysts said the rise was...
                                                       ...                                                 
2245    "" -- [MASK]-[MASK] [MASK] of the 's  -- is being used in a [MASK] for a bank and some [MASK] ar...
2246    , the [MASK] stock [MASK] [MASK] [MASK] to [MASK] [MASK] to [MASK] the market, has been [MASK] f...
2247    , 's biggest [MASK] and financial [MASK], is [MASK] for a [MASK] [MASK] when [MASK] [MASK] the [.

### Evaluate

In [ ]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=32)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    test_pred = []
    test_truth = []
    model.eval()
    
    with torch.no_grad():

        for test_input, test_label in test_dataloader:
            
            test_label = test_label.to(device)
            # mask = test_input['attention_mask'].to(device)
            input_id = test_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, test_label)

            test_pred.append(output.logits.argmax(dim=1))
            test_truth.append(test_label)
        
        test_truth = torch.cat(test_truth).detach().cpu().numpy()
        test_pred = torch.cat(test_pred).detach().cpu().numpy()
        test_acc = accuracy_score(test_truth, test_pred)
        test_f1 = f1_score(test_truth, test_pred, average='macro')
    
    print('Test Accuracy:  {:.4f} | Test F1:  {:.4f}'.format(test_acc, test_f1))
    return test_acc, test_f1
    
print(test_df['masked_content'])
evaluate(model, test_df)

### Fine-Tuning

In [ ]:
def fine_tuning(lr_list, epoch_list, batch_list, checkpoint_path='tuning2'):
    
    index_list, index2_list, index3_list = [], [], []
    train_accs, train_f1s = [], []
    test_accs, test_f1s = [], []
    
    for lr in lr_list:
        for epoch in epoch_list:
            for batch_size in batch_list:
            
                model = BertClassifier()
                # mask_contents(threshold=threshold)
                print('lr at {}, epoch at {}, batch_size at {}'.format(lr, epoch, batch_size))
                train_acc, train_f1, val_acc, val_f1 = train(model, train_df, valid_df, lr, epoch, batch_size)
                test_acc, test_f1 = evaluate(model, test_df)

                train_accs.append(train_acc)
                train_f1s.append(train_f1)
                test_accs.append(test_acc)
                test_f1s.append(test_f1)
                index_list.append(lr)
                index2_list.append(epoch)
                index3_list.append(batch_size)

                result_df = pd.DataFrame(index_list, columns=['lr'])
                result_df['epoch'] = index2_list
                result_df['batch_size'] = index3_list
                result_df['train_accs'] = train_accs
                result_df['train_f1s'] = train_f1s
                result_df['test_accs'] = test_accs
                result_df['test_f1s'] = test_f1s
                result_df['model_name'] = 'bert_uncased'
                result_df.to_csv(checkpoint_path + '.csv', index=False)
                # break
    return index_list, index2_list, index3_list, train_accs, train_f1s, test_accs, test_f1s

In [ ]:
# C50 [5e-5 + 10 + 16, 2e-5 + 15 + 16, 1e-5 + 15 + 8]
# All The News [2e-5 + 15 + 16]
lr_list = [2e-5] #[5e-5, 2e-5, 1e-5]
epoch_list = [15] #[10, 15]
batch_list = [16] #[16]
fine_tuning(lr_list=lr_list, epoch_list=epoch_list, batch_list=batch_list)

### Grid Search IDF

In [13]:
import warnings

def grid_search_idf(threshold_list, checkpoint_path='checkpoint_C502'):
    
    index_list = []
    train_accs, train_f1s = [], []
    test_accs, test_f1s = [], []
    
    for threshold in threshold_list:
        
        # warnings.filterwarnings(action='once')
        model = BertClassifier()
        mask_contents(threshold=threshold)
        print('idf removal threshold at', threshold)
        print(train_df['masked_content'])
        train_acc, train_f1, val_acc, val_f1 = train(model, train_df, valid_df, LR, EPOCHS, BATCH_SIZE)
        test_acc, test_f1 = evaluate(model, test_df)
        
        train_accs.append(train_acc)
        train_f1s.append(train_f1)
        test_accs.append(test_acc)
        test_f1s.append(test_f1)
        index_list.append(threshold)
        
        result_df = pd.DataFrame(index_list, columns=['idf_threshold'])
        result_df['train_accs'] = train_accs
        result_df['train_f1s'] = train_f1s
        result_df['test_accs'] = test_accs
        result_df['test_f1s'] = test_f1s
        result_df['model_name'] = 'bert_uncased'
        result_df.to_csv(checkpoint_path + '.csv', index=False)
        # break
    return index_list, train_accs, train_f1s, test_accs, test_f1s

In [ ]:
pd.options.display.max_colwidth = 100
EPOCHS = 15
LR = 2e-5
BATCH_SIZE = 16
grid_search_idf(threshold_list=np.arange(7.5, 1, -0.5))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

idf removal threshold at 7.5
0       The capital market, [MASK] criticised for a lack of transparency, needs an independent regulator...
1       beat the Republic 3-0 in a ice hockey game on , setting up a showdown for group [MASK] against a...
2       The fall in the trade deficit to crowns in from in buoyed market sentiment, and the goods import...
3       paper concern a.s. on said its net profit fell as a shutdown of its main paper mill, coupled wit...
4       shares rallied on following the coalition government's win in elections, but analysts said the r...
                                                       ...                                                 
2245    "" -- counter-culture [MASK] of the 's  -- is being used in a [MASK] for a bank and some fans ar...
2246    , the flamboyant stock picker hired recently to spearhead drive to dominate the market, has been...
2247    , 's biggest city and financial capital, is [MASK] for a near shutdown when protesters hit the s...

100%|█████████████████████████████████████████| 141/141 [01:54<00:00,  1.24it/s]


Epochs: 1 | Train Loss: 545.6220 | Train Accuracy: 0.0502 | Train F1: 0.0389
           | Val Loss:   58.0571 | Val Accuracy:   0.1560 | Val F1:   0.1045


100%|█████████████████████████████████████████| 141/141 [01:52<00:00,  1.26it/s]


...

100%|█████████████████████████████████████████| 141/141 [01:51<00:00,  1.26it/s]


Epochs: 12 | Train Loss: 126.5472 | Train Accuracy: 0.9351 | Train F1: 0.9331
           | Val Loss:   25.8264 | Val Accuracy:   0.5960 | Val F1:   0.5683


100%|█████████████████████████████████████████| 141/141 [01:52<00:00,  1.26it/s]


Epochs: 13 | Train Loss: 116.3483 | Train Accuracy: 0.9480 | Train F1: 0.9467
           | Val Loss:   25.1467 | Val Accuracy:   0.6160 | Val F1:   0.5917


 81%|█████████████████████████████████▏       | 114/141 [01:30<00:21,  1.26it/s]

### Saliency Map

In [ ]:
def get_saliency_map(model, df):
    input_text = [tokenizer(text, padding='max_length', max_length = 512, truncation=True, return_tensors="pt") for text in df['content']]
    input = input_text[0]
    print(input.shape)
    for param in model.parameters():
        param.requires_grad = False
    model.eval()
    # input.unsqueeze_(0)
    input.requires_grad = True
    preds = model(input)
    score, indices = torch.max(preds, 1)
    #backward pass to get gradients of score predicted class w.r.t. input image
    score.backward()
    #get max along channel axis
    slc, _ = torch.max(torch.abs(input.grad[0]), dim=0)
    #normalize to [0..1]
    slc = (slc - slc.min())/(slc.max()-slc.min())
    return slc

input_slc = get_saliency_map(model, train_df)
print(input_slc.shape)
input_slc

### SHAP

In [ ]:
import shap
import scipy as sp

def f(x):
    tv = torch.tensor([tokenizer.encode(v, padding='max_length', \
        max_length=512, truncation=True) for v in x]).cuda()
    outputs = model(tv)[0].detach().cpu().numpy()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
    val = sp.special.logit(scores[:,1]) # use one vs rest logit units
    return val

explainer = shap.Explainer(f, tokenizer)
shap_values = explainer(train_df['masked_content'][:3], fixed_context=1, batch_size=32)

In [ ]:
shap.plots.bar(shap_values.abs.mean(0)) # default

In [ ]:
shap.plots.bar(shap_values.abs.sum(0))

In [ ]:
shap.plots.bar(shap_values.abs.max(0))

In [ ]:
shap.plots.text(shap_values)